In [1]:
import cv2
import os
import datetime
import pandas
from openalea.phenomenal.calibration import CalibrationCamera


PREFIX='/ifb/data/public/teachdata/reprohack3/ARCH2016-04-15/binaries'

## read index
def read_index(prefix=os.path.join(PREFIX,'..')):
	df = pandas.read_csv(os.path.join(prefix, 'ZA16_organized_database.csv'),sep=';')
	for col in ('path_http','camera_angle', 'dates', 'view_type'):
		df.loc[:,col] = map(eval, df.loc[:,col])
	return df

## binary path
def get_remote_path_bin(row, prefix=PREFIX, delta=0):
    pot_number = int(row["plant"][:4])

    date = datetime.datetime.strptime(row["date"], "%Y-%m-%d %H:%M:%S.%f")
    date = date + datetime.timedelta(seconds=delta)
    str_date = date.strftime('%Y-%m-%d_%H-%M-%S')

    str_views = ['sv' if v == 'side' else 'tv' for v in row["view_type"]]

    filenames = list()
    for filename, view, angle, date in zip(row["path_http"],
                                           str_views,
                                           row["camera_angle"],
                                           row["dates"]):

        r = filename.split('/')


        # ZA16
        f = (prefix+
            '/task_{task}/'
            't-{task}_p-{pot}_{view}{angle}_{date}_bin.png'.format(
                name_exp=r[5],
                task=r[6],
                pot=pot_number,
                view=view,
                angle=angle,
                date=str_date))

        filenames.append(f)

    return filenames


def get_bin_images(row, prefix=PREFIX):
	#find filenames
	for delta in [0, -1, 1, -2, 2, -3, 3, -4, 4]:
		filenames = get_remote_path_bin(row, prefix=prefix,delta=delta)
		if os.path.exists(filenames[0]):
			break
	if delta == 4:
		raise ValueError('Can not find binaries')
	imdict={t:{} for t in set(row.view_type)}
	for view, angle, path in zip(row.view_type,row.camera_angle,filenames):
		imdict[view][angle]=cv2.imread(path, cv2.IMREAD_UNCHANGED)
	return imdict


def get_calibrations(row, prefix=os.path.join(PREFIX,'..','calibration')):
	calib = {}
	for view in set(row.view_type):
		path = os.path.join(prefix, (row.cabin + '_' + view + '_1_1_wide.json'))
		calib[view] = CalibrationCamera.load(path)
	return calib

# Plant growth animation

In [31]:
%matplotlib notebook

import cv2

import openalea.phenomenal.data as phm_data 
import openalea.phenomenal.display as phm_display
import openalea.phenomenal.object as phm_obj
import openalea.phenomenal.multi_view_reconstruction as phm_mvr
import openalea.phenomenal.mesh as phm_mesh
import openalea.phenomenal.display.notebook as phm_display_notebook

## Load data

In [2]:
df = read_index()
df.head()

,cabin,camera_angle,date,dates,daydate,experiment,genotype,nview,path_http,plant,shooting_frame,task,timestamp,uri,view_type,pos,line
0,elcom_c1,"[330, 300, 270, 240, 210, 150, 180, 90, 120, 3...",2016-05-18 04:30:13.018825,"[2016-05-18 04:30:13.018825, 2016-05-18 04:30:...",18/05/2016,ZA16,Lo1270_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0001/Lo1270_H/ZM2887/d151/WD/3/01_01/ARCH2016-...,elcom_2_c1_wide,5236,1463538613,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",1,1
1,elcom_c2,"[300, 330, 240, 270, 180, 210, 120, 150, 60, 9...",2016-05-19 03:50:35.300685,"[2016-05-19 03:50:35.300685, 2016-05-19 03:50:...",19/05/2016,ZA16,Lo1270_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0001/Lo1270_H/ZM2887/d151/WD/3/01_01/ARCH2016-...,elcom_2_c2_wide,5240,1463622635,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",1,1
2,elcom_c2,"[330, 270, 300, 210, 240, 150, 180, 90, 120, 3...",2016-05-20 06:52:56.492283,"[2016-05-20 06:52:56.492283, 2016-05-20 06:52:...",20/05/2016,ZA16,Lo1270_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0001/Lo1270_H/ZM2887/d151/WD/3/01_01/ARCH2016-...,elcom_2_c2_wide,5248,1463719976,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",1,1
3,elcom_c2,"[330, 270, 300, 210, 240, 150, 180, 90, 120, 3...",2016-05-22 04:05:34.442505,"[2016-05-22 04:05:34.442505, 2016-05-22 04:05:...",22/05/2016,ZA16,Lo1270_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0001/Lo1270_H/ZM2887/d151/WD/3/01_01/ARCH2016-...,elcom_2_c2_wide,5269,1463882734,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",1,1
4,elcom_c2,"[330, 270, 300, 210, 240, 150, 180, 90, 120, 3...",2016-05-23 04:05:23.606912,"[2016-05-23 04:05:23.606912, 2016-05-23 04:05:...",23/05/2016,ZA16,Lo1270_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0001/Lo1270_H/ZM2887/d151/WD/3/01_01/ARCH2016-...,elcom_2_c2_wide,5270,1463969123,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",1,1


In [4]:
df.shape[0]

57437

## Select 1 plant

In [19]:
dfl = df[df.plant == df.plant[1500]]

In [20]:
dfl.shape[0]

26

In [25]:
dfl = dfl.sort_values(['date'])

In [26]:
dfl.head()

,cabin,camera_angle,date,dates,daydate,experiment,genotype,nview,path_http,plant,shooting_frame,task,timestamp,uri,view_type,pos,line
1481,elcom_c2,"[330, 270, 300, 210, 240, 150, 180, 120, 90, 6...",2016-05-18 04:52:56.427868,"[2016-05-18 04:52:56.427868, 2016-05-18 04:52:...",18/05/2016,ZA16,DKMBST_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0058/DKMBST_H/ZM2800/d89/WD/3/01_58/ARCH2016-0...,elcom_2_c2_wide,5236,1463539976,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",58,1
1482,elcom_c2,"[330, 270, 300, 210, 240, 150, 180, 90, 120, 3...",2016-05-19 04:13:22.755899,"[2016-05-19 04:13:22.755899, 2016-05-19 04:13:...",19/05/2016,ZA16,DKMBST_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0058/DKMBST_H/ZM2800/d89/WD/3/01_58/ARCH2016-0...,elcom_2_c2_wide,5240,1463624002,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",58,1
1483,elcom_c1,"[300, 330, 240, 270, 180, 210, 120, 150, 60, 9...",2016-05-20 07:14:35.574526,"[2016-05-20 07:14:35.574526, 2016-05-20 07:14:...",20/05/2016,ZA16,DKMBST_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0058/DKMBST_H/ZM2800/d89/WD/3/01_58/ARCH2016-0...,elcom_2_c1_wide,5248,1463721275,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",58,1
1484,elcom_c1,"[300, 330, 240, 270, 180, 210, 120, 150, 60, 9...",2016-05-22 04:27:12.565693,"[2016-05-22 04:27:12.565693, 2016-05-22 04:27:...",22/05/2016,ZA16,DKMBST_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0058/DKMBST_H/ZM2800/d89/WD/3/01_58/ARCH2016-0...,elcom_2_c1_wide,5269,1463884032,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",58,1
1485,elcom_c1,"[300, 330, 240, 270, 180, 210, 120, 150, 60, 9...",2016-05-23 04:26:54.472456,"[2016-05-23 04:26:54.472456, 2016-05-23 04:26:...",23/05/2016,ZA16,DKMBST_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0058/DKMBST_H/ZM2800/d89/WD/3/01_58/ARCH2016-0...,elcom_2_c1_wide,5270,1463970414,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",58,1
1486,elcom_c1,"[300, 330, 240, 270, 180, 210, 120, 150, 60, 9...",2016-05-25 04:26:13.629714,"[2016-05-25 04:26:13.629714, 2016-05-25 04:26:...",25/05/2016,ZA16,DKMBST_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0058/DKMBST_H/ZM2800/d89/WD/3/01_58/ARCH2016-0...,elcom_2_c1_wide,5278,1464143173,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",58,1
1487,elcom_c1,"[330, 270, 300, 210, 240, 150, 180, 90, 120, 3...",2016-05-27 04:58:38.249535,"[2016-05-27 04:58:38.249535, 2016-05-27 04:58:...",27/05/2016,ZA16,DKMBST_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0058/DKMBST_H/ZM2800/d89/WD/3/01_58/ARCH2016-0...,elcom_2_c1_wide,5292,1464317918,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",58,1
1488,elcom_c1,"[330, 270, 300, 210, 240, 150, 180, 90, 120, 3...",2016-05-29 04:26:52.796144,"[2016-05-29 04:26:52.796144, 2016-05-29 04:26:...",29/05/2016,ZA16,DKMBST_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0058/DKMBST_H/ZM2800/d89/WD/3/01_58/ARCH2016-0...,elcom_2_c1_wide,5302,1464488812,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",58,1
1489,elcom_c1,"[300, 330, 240, 270, 180, 210, 120, 150, 60, 9...",2016-05-30 04:27:15.093686,"[2016-05-30 04:27:15.093686, 2016-05-30 04:27:...",30/05/2016,ZA16,DKMBST_H,13,[http://stck-lepse.supagro.inra.fr/phenoarch/r...,0058/DKMBST_H/ZM2800/d89/WD/3/01_58/ARCH2016-0...,elcom_2_c1_wide,5307,1464575235,[u'http://www.phenome-fppn.fr/m3p/arch/2016/ic...,"[side, side, side, side, side, side, side, sid...",58,1
1490,elcom_c1,"[300, 330, 240, 270, 180, 210, 120, 150, 60, 9...",2016-06-01 04:30:09.673266,"[2016-06-01 04:30:09.673266, 2016-06-01 04:30:...",01/06/2016,ZA

In [28]:
def routine_select_ref_angle(bin_side_images):

    max_len = 0
    max_angle = None
    
    for angle in bin_side_images:
    
        x_pos, y_pos, x_len, y_len = cv2.boundingRect(cv2.findNonZero(bin_side_images[angle]))

        if x_len > max_len:
            max_len = x_len
            max_angle = angle

    return max_angle

In [33]:
voxels_size = 8 # mm
error_tolerance = 2
plants_voxels = []
for plant_number in range(0, dfl.shape[0]):
    rpd = get_remote_path_bin(df.loc[plant_number])
    calibrations = get_calibrations(row=df.loc[plant_number])
    bin_images = get_bin_images(row=df.loc[plant_number])
    
    refs_angle_list = [routine_select_ref_angle(bin_images["side"])]

    image_views = list()
    for id_camera in bin_images:
        for angle in bin_images[id_camera]:
            projection = calibrations[id_camera].get_projection(angle)

            image_ref = None
            if id_camera == "side" and angle in refs_angle_list:
                image_ref = bin_images[id_camera][angle]

            inclusive = False
            if id_camera == "top":
                inclusive = True

            image_views.append(phm_obj.ImageView(
                bin_images[id_camera][angle], 
                projection, 
                inclusive=inclusive, 
                image_ref=image_ref))
            
    plants_voxels.append(
        phm_mvr.reconstruction_3d(
            image_views, 
            voxels_size=voxels_size,
            error_tolerance=error_tolerance)
    )

## Test voxels

In [34]:
phm_display_notebook.show_voxel_grid(plants_voxels[0], size=1)

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMi4wLCAxLjIyNDY0Njc5OTE0NzM1MzJlLTE2KSwgcXVhdGVybmnigKY=


In [43]:
phm_display_notebook.show_voxel_grid(plants_voxels[25], size=1)

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMi4wLCAxLjIyNDY0Njc5OTE0NzM1MzJlLTE2KSwgcXVhdGVybmnigKY=


## Serialize 

In [37]:
for i, voxel_grid in enumerate(plants_voxels):
    voxel_grid.write("voxel_{}_size_{}.npz".format(i, voxels_size))

### Build meshes

In [38]:
meshes = []
for i, voxel_grid in enumerate(plants_voxels):
    vertices, faces = phm_mesh.meshing(
        voxel_grid.to_image_3d(),
        reduction=0.90,
        smoothing_iteration=5,
        verbose=True
    )
    meshes.append(
        dict(vertices=vertices, faces=faces)
    )

Marching cubes : 
	Iso value :0.5

	There are 498 points.
	There are 955 polygons.
Smoothing : 
	Feature angle :120.0
	Number of iteration :5
	Pass band : 0.01

Decimation : 
	Reduction (percentage) :0.9

	Before decimation
	-----------------
	There are 498 points.
	There are 955 polygons.

	After decimation
	-----------------
	There are 0.9 points.
	There are 498 polygons.
Marching cubes : 
	Iso value :0.5

	There are 702 points.
	There are 1358 polygons.
Smoothing : 
	Feature angle :120.0
	Number of iteration :5
	Pass band : 0.01

Decimation : 
	Reduction (percentage) :0.9

	Before decimation
	-----------------
	There are 702 points.
	There are 1358 polygons.

	After decimation
	-----------------
	There are 0.9 points.
	There are 702 polygons.
Marching cubes : 
	Iso value :0.5

	There are 887 points.
	There are 1736 polygons.
Smoothing : 
	Feature angle :120.0
	Number of iteration :5
	Pass band : 0.01

Decimation : 
	Reduction (percentage) :0.9

	Before decimation
	-----------------


Marching cubes : 
	Iso value :0.5

	There are 10591 points.
	There are 21076 polygons.
Smoothing : 
	Feature angle :120.0
	Number of iteration :5
	Pass band : 0.01

Decimation : 
	Reduction (percentage) :0.9

	Before decimation
	-----------------
	There are 10591 points.
	There are 21076 polygons.

	After decimation
	-----------------
	There are 0.9 points.
	There are 10591 polygons.
Marching cubes : 
	Iso value :0.5

	There are 12560 points.
	There are 24984 polygons.
Smoothing : 
	Feature angle :120.0
	Number of iteration :5
	Pass band : 0.01

Decimation : 
	Reduction (percentage) :0.9

	Before decimation
	-----------------
	There are 12560 points.
	There are 24984 polygons.

	After decimation
	-----------------
	There are 0.9 points.
	There are 12560 polygons.
Marching cubes : 
	Iso value :0.5

	There are 15835 points.
	There are 31504 polygons.
Smoothing : 
	Feature angle :120.0
	Number of iteration :5
	Pass band : 0.01

Decimation : 
	Reduction (percentage) :0.9

	Before decimatio

	After decimation
	-----------------
	There are 0.9 points.
	There are 23378 polygons.
Marching cubes : 
	Iso value :0.5

	There are 25997 points.
	There are 51724 polygons.
Smoothing : 
	Feature angle :120.0
	Number of iteration :5
	Pass band : 0.01

Decimation : 
	Reduction (percentage) :0.9

	Before decimation
	-----------------
	There are 25997 points.
	There are 51724 polygons.

	After decimation
	-----------------
	There are 0.9 points.
	There are 25997 polygons.
Marching cubes : 
	Iso value :0.5

	There are 27756 points.
	There are 55312 polygons.
Smoothing : 
	Feature angle :120.0
	Number of iteration :5
	Pass band : 0.01

Decimation : 
	Reduction (percentage) :0.9

	Before decimation
	-----------------
	There are 27756 points.
	There are 55312 polygons.

	After decimation
	-----------------
	There are 0.9 points.
	There are 27756 polygons.
Marching cubes : 
	Iso value :0.5

	There are 29946 points.
	There are 59678 polygons.
Smoothing : 
	Feature angle :120.0
	Number of iterat

In [39]:
phm_display_notebook.show_mesh(meshes[0]['vertices'], meshes[0]['faces'])

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMi4wLCAxLjIyNDY0Njc5OTE0NzM1MzJlLTE2KSwgcXVhdGVybmnigKY=


In [68]:
phm_display_notebook.show_mesh(meshes[25]['vertices'], meshes[25]['faces'])

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMi4wLCAxLjIyNDY0Njc5OTE0NzM1MzJlLTE2KSwgcXVhdGVybmnigKY=


In [41]:
import ipyvolume

### Test ipyvolume plotting

In [72]:
x, y, z = (plants_voxels[25].voxels_position[:, 0],
           plants_voxels[25].voxels_position[:, 1],
           plants_voxels[25].voxels_position[:, 2])

ipyvolume.scatter(x, y, z, size=2, marker="box", color="green")

x_min = plants_voxels[25].voxels_position[:, 0].min()
x_max = plants_voxels[25].voxels_position[:, 0].max()
y_min = plants_voxels[25].voxels_position[:, 1].min()
y_max = plants_voxels[25].voxels_position[:, 1].max()
z_min = plants_voxels[25].voxels_position[:, 2].min()
z_max = plants_voxels[25].voxels_position[:, 2].max()

xyz_max = max(x_max - x_min, y_max - y_min, z_max - z_min)
ipyvolume.xlim(x_min, x_min + xyz_max)
ipyvolume.ylim(y_min, y_min + xyz_max)
ipyvolume.zlim(z_min, z_min + xyz_max)
ipyvolume.view(10, 10)
ipyvolume.show()

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgbWF0cml4V29ybGROZWVkc1VwZGF0ZT1UcnVlLCBwb3NpdGlvbj0oMC4zNTg3MjY4NTUyODjigKY=


In [49]:
plants_voxels[25].voxels_position[:, 0]

array([  68.,   44., -412., ...,  636.,  284.,  -36.])

In [50]:
len(plants_voxels[25].voxels_position[:, 0])

32040

In [51]:
len(plants_voxels[0].voxels_position[:, 0])

231

In [73]:
max_voxel_vertices = 0
for voxel_grid in plants_voxels:    
    if len(voxel_grid.voxels_position[:, 0]) > max_voxel_vertices:
        max_voxel_vertices = len(voxel_grid.voxels_position[:, 0])
max_voxel_vertices

32040

### Build animation voxels

In [149]:
import numpy as np

x_start = plants_voxels[0].voxels_position[0, 0]
y_start = plants_voxels[0].voxels_position[0, 1]
z_start = plants_voxels[0].voxels_position[0, 2]

x = [[x_start for i in range(0, max_voxel_vertices)] for _ in range(0, len(plants_voxels))]
y = [[y_start for i in range(0, max_voxel_vertices)] for _ in range(0, len(plants_voxels))]
z = [[z_start for i in range(0, max_voxel_vertices)] for _ in range(0, len(plants_voxels))]

for i, voxel_grid in enumerate(plants_voxels):
    xs = voxel_grid.voxels_position[:, 0]
    ys = voxel_grid.voxels_position[:, 1]
    zs = voxel_grid.voxels_position[:, 2]
    
    #pts = [[xs[k], ys[k], zs[k]] for k in range(len(voxel_grid.voxels_position[:, 0]))]
    #pts = sorted(pts, key =  lambda x: x[0]*x[0] + x[1]*x[1] +x[2]*x[2])
    
    #xs = []
    #ys = []
    #zs = []
    #for v in pts:
    #    xs.append(v[0])
    #    ys.append(v[1])
    #    zs.append(v[2])
    
    xs, ys, zs = zip(*sorted(zip(xs, ys, zs)))
    
    for j, xi in enumerate(xs):
        x[i][j] = xi 
    
    for j, yi in enumerate(ys):
        y[i][j] = yi
    
    for j, zi in enumerate(zs):
        z[i][j] = zi
        
x = np.array(x)
y = np.array(y)
z = np.array(z)

### Build animaton

In [150]:
ipyvolume.figure()
s = ipyvolume.scatter(
    x, y, z, 
    color="green", 
    visible=True,
    marker='sphere', 
    size=1, 
    
    grow_limits=False
)
ipyvolume.xyzlim(-1, 1)
ipyvolume.animation_control(
    s, 
    interval=1000
) # shows controls for animation controls

x_min = plants_voxels[25].voxels_position[:, 0].min()
x_max = plants_voxels[25].voxels_position[:, 0].max()
y_min = plants_voxels[25].voxels_position[:, 1].min()
y_max = plants_voxels[25].voxels_position[:, 1].max()
z_min = plants_voxels[25].voxels_position[:, 2].min()
z_max = plants_voxels[25].voxels_position[:, 2].max()

xyz_max = max(x_max - x_min, y_max - y_min, z_max - z_min)
ipyvolume.xlim(x_min, x_min + xyz_max)
ipyvolume.ylim(y_min, y_min + xyz_max)
ipyvolume.zlim(z_min, z_min + xyz_max)

ipyvolume.show()

VkJveChjaGlsZHJlbj0oRmlndXJlKGNhbWVyYT1QZXJzcGVjdGl2ZUNhbWVyYShmb3Y9NDYuMCwgcG9zaXRpb249KDAuMCwgMC4wLCAyLjApLCBxdWF0ZXJuaW9uPSgwLjAsIDAuMCwgMC4wLCDigKY=
